#ETL Pipeline

This project extracts, transforms, and loads data on the top 10 largest banks by market capitalization from Wikipedia. The data is transformed into multiple currencies and stored in both CSV and SQLite database formats, with functionality to execute SQL queries for further analysis.

## Installations

In [7]:
# !pip install requests
# !pip install pandas
# !pip install beautifulsoup4
# !pip install lxml
# !pip install numpy

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 

In [8]:
import requests
from datetime import datetime
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import sqlite3

## Log progress function

In [9]:
# Task 1: Log Progress Function
def log_progress(message):
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    with open('code_log.txt', 'a') as log_file:
        log_file.write(f'{timestamp} : {message}\n')

## extraction

In [10]:
# Task 2: Data Extraction
def extract(url, table_attribs):
    response = requests.get(url)
    if response.status_code != 200:
        log_progress(f"Failed to retrieve the page. Status code: {response.status_code}")
        return None

    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', {'class': table_attribs})
    if table is None:
        log_progress(f"Table with class {table_attribs} not found")
        return None

    rows = table.find_all('tr')
    data = []

    for row in rows[1:11]:  # Extracting top 10 rows
        cols = row.find_all('td')
        if len(cols) > 2:  # Ensure there are enough columns
            bank_name = cols[1].find_all('a')[1]['title']
            market_cap = float(cols[2].text.strip().replace('\n', '').replace(',', ''))
            data.append({'Name': bank_name, 'MC_USD_Billion': market_cap})

    df = pd.DataFrame(data)
    log_progress("Data extraction complete. Initiating Transformation process")
    return df


##Transformation

In [11]:
# Task 3: Data Transformation
def transform(df, csv_path):
    log_progress("Starting data transformation...")
    exchange_rates = pd.read_csv(csv_path)
    exchange_rate = dict(zip(exchange_rates['Currency'], exchange_rates['Rate']))

    df['MC_GBP_Billion'] = [np.round(x * exchange_rate['GBP'], 2) for x in df['MC_USD_Billion']]
    df['MC_EUR_Billion'] = [np.round(x * exchange_rate['EUR'], 2) for x in df['MC_USD_Billion']]
    df['MC_INR_Billion'] = [np.round(x * exchange_rate['INR'], 2) for x in df['MC_USD_Billion']]

    log_progress("Data transformation complete. Initiating Loading process")
    return df

##Storage

In [12]:
# Task 4: Load to CSV
def load_to_csv(df, output_path):
    log_progress("Saving data to CSV file...")
    df.to_csv(output_path, index=False)
    log_progress("Data saved to CSV file")

# Task 5: Load to Database
def load_to_db(df, db_name, table_name):
    log_progress("Saving data to database...")
    conn = sqlite3.connect(db_name)
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.close()
    log_progress("Data saved to database")

##Queries

In [13]:
# Task 6: Run Queries
def run_queries(query_statement, conn):
    log_progress(f"Running query: {query_statement}")
    cursor = conn.cursor()
    cursor.execute(query_statement)
    results = cursor.fetchall()
    print(f"Query: {query_statement}")
    for row in results:
        print(row)
    log_progress("Query execution complete")

#Run the pipeline

In [14]:
# Main function
def main():
    url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
    table_attribs = 'wikitable'
    df = extract(url, table_attribs)

    if df is not None:
        csv_path = 'exchange_rate.csv'
        df = transform(df, csv_path)
        output_csv_path = './Largest_banks_data.csv'
        load_to_csv(df, output_csv_path)

        db_name = 'Banks.db'
        table_name = 'Largest_banks'
        load_to_db(df, db_name, table_name)

        conn = sqlite3.connect(db_name)

        # Execute queries
        run_queries("SELECT * FROM Largest_banks", conn)
        run_queries("SELECT AVG(MC_USD_Billion) FROM Largest_banks", conn)
        run_queries("SELECT Name FROM Largest_banks LIMIT 5", conn)

        conn.close()

        print(df)
        print(df['MC_EUR_Billion'][4])  # Print the market capitalization of the 5th largest bank in billion EUR

if __name__ == "__main__":
    main()


Query: SELECT * FROM Largest_banks
('JPMorgan Chase', 432.92, 346.34, 402.62, 35910.71)
('Bank of America', 231.52, 185.22, 215.31, 19204.58)
('Industrial and Commercial Bank of China', 194.56, 155.65, 180.94, 16138.75)
('Agricultural Bank of China', 160.68, 128.54, 149.43, 13328.41)
('HDFC Bank', 157.91, 126.33, 146.86, 13098.63)
('Wells Fargo', 155.87, 124.7, 144.96, 12929.42)
('HSBC', 148.9, 119.12, 138.48, 12351.26)
('Morgan Stanley', 140.83, 112.66, 130.97, 11681.85)
('China Construction Bank', 139.82, 111.86, 130.03, 11598.07)
('Bank of China', 136.81, 109.45, 127.23, 11348.39)
Query: SELECT AVG(MC_USD_Billion) FROM Largest_banks
(189.982,)
Query: SELECT Name FROM Largest_banks LIMIT 5
('JPMorgan Chase',)
('Bank of America',)
('Industrial and Commercial Bank of China',)
('Agricultural Bank of China',)
('HDFC Bank',)
                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1         